# ML Course 3: Classification: Ass Week 1
## Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started! see also https://github.com/smarthi/UnivOfWashington-Machine-Learning/blob/master/Classification/week1/module-2-linear-classifier-assignment-blank.ipynb
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [171]:
from __future__ import division
import sframe as sf
import pandas as pd
import numpy as np
import math
import string

# Data preperation

**1.** Load the dataset consisting of baby product reviews on Amazon.com. Store the data in a data frame products.

In [150]:
products = sf.SFrame('../../course-1/Week3/amazon_baby.csv')
#dtype_dict = {'name':str, 'review':str, 'rating':int}
#products = pd.read_csv('../../course-1/Week3/amazon_baby.csv', dtype = dtype_dict)

Finished parsing file /mnt/win-data/shared/stats-R/ml_specialization/machine-learning-specialization/course-1/Week3/amazon_baby.csv

Parsing completed. Parsed 100 lines in 0.260428 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 105270 lines. Lines per second: 234910

Finished parsing file /mnt/win-data/shared/stats-R/ml_specialization/machine-learning-specialization/course-1/Week3/amazon_baby.csv

Parsing completed. Parsed 183531 lines in 0.624535 secs.

Now, let us see a preview of what the dataset looks like.

In [152]:
#print "Features are ",products.columns
#print "Number of features = %d" %len(products.columns)
#(num_rows,num_cols) = products.shape
#print "Number of data items = %d" %products.size
#print products.ndim
#print products.index
#print "Number of samples/rows = %d and num of features/cols =  %d" %products.shape
products.head(2)

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [153]:
#products.describe()
#print products.iloc[269]   #iloc: Purely integer-location based indexing for selection by position
#products[269:270] 

**2.** Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality, so that words "cake." and "cake!" are counted as the same word.
2. Apply this function to every element in the review column of products, and save the result to a new column review_clean. (Transform the reviews into word-counts. - not doing this here)

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

IMPORTANT. Make sure to fill n/a values in the review column with empty strings (if applicable). The n/a values indicate empty reviews. For instance, Pandas's the fillna() method lets you replace all N/A's in the review columns as follows:

In [62]:
#products = products.fillna({'review':''})  # fill in N/A's in the review column

In [154]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

products['review_clean'] = products['review'].apply(remove_punctuation)
#review_without_puctuation = products['review'].apply(remove_punctuation)
#products['word_count'] = sf.text_analytics.count_words(review_without_puctuation)

In [155]:
#print products.dtypes
#products['review'] = products['review'].astype(str) 

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [156]:
#products[269:270]['word_count']
#products[269:270]
#products = products.fillna({'review':''})  # fill in N/A's in the review column

## Extract sentiments

**3.** We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment. 

In [157]:
print len(products)
products = products[ products['rating'] != 3 ]
print len(products)
products.head(2)

183531
166752


name,review,rating,review_clean
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,it came early and was notdisappointed i love ...
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,Very soft and comfortableand warmer than it ...


**4.** Now, we will assign reviews with a rating of 4 or higher to be positive reviews, while the ones with rating of 2 or lower are negative. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label. A good way is to create an anonymous function that converts a rating into a class label and then apply that function to every element in the rating column. In SFrame, you would use apply():

In [158]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
len(products) == len ( products[products['sentiment'] == -1]) + \
                len ( products[products['sentiment'] == 1])

True

In [159]:
products.head(2)

name,review,rating,review_clean,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,it came early and was notdisappointed i love ...,1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,Very soft and comfortableand warmer than it ...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1). We can also see that all product reviews got a sentiment (+1 or -1) cause the sum of -1 and +1 sentiments add up to number of product reviews we have.

## Split data into training and test sets

**5.** Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result. 

We are not using SFrame, so download the list of indices for the training and test sets: module-2-assignment-train-idx.json, module-2-assignment-test-idx.json. IMPORTANT: If you are using a programming language with 1-based indexing (e.g. R, Matlab), make sure to increment all indices by 1.

In [160]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)  #133416
print len(test_data)   #33336

'''import re
#https://automatetheboringstuff.com/chapter14/
each file is in thie format 
[8, 9, 14, 18, 24, 31, 32, 36, 38, 50, ....]

train_file = open("module-2-assignment-train-idx.json", "r")
train = train_file.read()
train = train[1:-1]    #strip first and last char '[' and ']'
train = list(train)
print train

#train = train.split(',')
print train
train_data = products.iloc[train,:]
print train_data.shape()

test_file = open("module-2-assignment-test-idx.json", "r")
test = test_file.read()
train_file.close()
test_file.close()

train_data = products.iloc[train,:]
print train_data.shape()
test_data =  products.iloc[test,:]
print len(test_data) 
'''

133416
33336


'import re\n#https://automatetheboringstuff.com/chapter14/\neach file is in thie format \n[8, 9, 14, 18, 24, 31, 32, 36, 38, 50, ....]\n\ntrain_file = open("module-2-assignment-train-idx.json", "r")\ntrain = train_file.read()\ntrain = train[1:-1]    #strip first and last char \'[\' and \']\'\ntrain = list(train)\nprint train\n\n#train = train.split(\',\')\nprint train\ntrain_data = products.iloc[train,:]\nprint train_data.shape()\n\ntest_file = open("module-2-assignment-test-idx.json", "r")\ntest = test_file.read()\ntrain_file.close()\ntest_file.close()\n\ntrain_data = products.iloc[train,:]\nprint train_data.shape()\ntest_data =  products.iloc[test,:]\nprint len(test_data) \n'

### Build the word count vector for each review

**6.** We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

*    Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
*    Compute the occurrences of the words in each review and collect them into a row vector.
*    Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix.
*    Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.

The following cell uses CountVectorizer in scikit-learn. Notice the token_pattern argument in the constructor.

In [161]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. 

Instructions for graphlab --> This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**7.** Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.

**Note:** This line may take 1-2 minutes.

In [165]:
from sklearn.linear_model import LogisticRegression
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
X = train_matrix
y = train_data['sentiment'].to_numpy()
sentiment_model = model.fit(X, y)

# check the accuracy on the training set
sentiment_model.score(X, y)

'''sentiment_model = graphlab.logistic_classifier.create(train_data, target = 'sentiment',
                                            features=['word_count'], validation_set=None)
'''                                                      

"sentiment_model = graphlab.logistic_classifier.create(train_data,\n                                                      target = 'sentiment',\n                                                      features=['word_count'],\n                                                      validation_set=None)\n"

In [180]:
sentiment_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

**8.** There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients. 

In [177]:
print sentiment_model.coef_
print "Non zero coefficients (including intercept)" , \
np.count_nonzero(sentiment_model.coef_) + np.count_nonzero(sentiment_model.intercept_)
#len(model_all.sparse_coef_)
print "(sentiment_model.coef_).shape[0]", (sentiment_model.coef_).shape[0]
#print sentiment_model.coef_.getnnz()
print "sentiment_model.coef_.shape:", sentiment_model.coef_.shape

[[ -1.24016858e+00   2.97817122e-05   2.75239185e-02 ...,   1.26002631e-02
    2.71201093e-03  -4.16780197e-05]]
Non zero coefficients (including intercept) 121713
(sentiment_model.coef_).shape[0] 1
sentiment_model.coef_.shape: (1, 121712)


In [181]:
# examine the coefficients - can't do sparse matrix X has no column names
# pd.DataFrame(zip(X.columns, np.transpose(sentiment_model.coef_))).head()

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [210]:
np1 = sum(1 for x in sentiment_model.coef_[0] if x > 0)
np2 = len([x for x in sentiment_model.coef_[0] if x > 0])
print np1
print np2
count_pos = 0
count_neg = 0
for i in range(len(sentiment_model.coef_[0])):
    if sentiment_model.coef_[0][i] >= 0:
        count_pos=count_pos+1
    else:
        count_neg=count_neg+1

print count_pos + count_neg
print "Number of positive weights: %s " % count_pos
print "Number of negative weights: %s " % count_neg
#print "Total: %s " % (num_positive_weights + num_negative_weights)

86906
86906
121712
Number of positive weights: 86906 
Number of negative weights: 34806 


**Quiz question:** How many weights are >= 0?  68419

## Making predictions with logistic regression
**9.** Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 data points/examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**. Take the 11th, 12th, and 13th data points in the test data and save them to sample_test_data.

In [211]:
sample_test_data = test_data[10:13]  #samples/rows 10,11,12
print sample_test_data['rating']
sample_test_data

[5, 2, 1]


name,review,rating,review_clean,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5,Absolutely love it andall of the Scripture in ...,1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2,Would not purchase againor recommend The decals ...,-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1,Was so excited to getthis product for my baby ...,-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [212]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [213]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

**10.** We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**. Note that scores is just a dot product of weights and features as in regression!! - so predicted values.

Hint: need to convert sample_test_data into the sparse matrix format first.

In [231]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

[  5.60224818  -3.17079879 -10.42488847]


In [57]:
# margin classfier --> margin=score, 
# score >= 0 on one side of margin, score<0 on other side of margin or decision boundary
#scores = sentiment_model.predict(sample_test_data, output_type='margin')
#print scores   
#print scores.dtype

[6.7346197270594494, -5.734130996760403, -14.668460404468794]
<bound method SArray.dtype of dtype: float
Rows: 3
[6.7346197270594494, -5.734130996760403, -14.668460404468794]>


### Predicting sentiment

**11.** These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions: 

Again there are many ways to skin the goat!!! Given scores is just a list of values, we can either use an in place lambda function of list comprehension to do this in one line, show off both ways below. Traditioanlly we would iterate through the list and pick up item/value one by one, inspect if its neg or pos and the store/print corresponding -1 or +1. In both cases a list of outcomes is produced.


In [217]:
#'numpy.ndarray' object has no attribute 'apply'
#predictions = scores.apply(lambda x : +1 if x > 0 else -1)  #lambda in place function
#print predictions                                           #scores is SArray.dtype: float

In [232]:
#list comprehension  - optimized iterator through values in list, for each item do test
predictions = [+1 if x > 0 else -1 for x in scores]    
print predictions

[1, -1, -1]


In [233]:
def predict_class(model, data):
    scores = model.decision_function(data)
    score_class = [+1 if x > 0 else -1 for x in scores]
    return (score_class)  #if no enclose return, prints None!!

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [234]:
print "My class predictions:"
print predict_class(sentiment_model, sample_test_matrix )
print "Class predictions according to  scikit-learn:" 
print sentiment_model.predict(sample_test_matrix)

My class predictions:
[1, -1, -1]
Class predictions according to  scikit-learn:
[ 1 -1 -1]


**Checkpoint**: Make sure your class predictions match with the ones obtained from sentiment_model. The logistic regression classifier in scikit-learn comes with the predict function for this purpose.

### Probability predictions

**12.** Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{Score})}.
$$ $$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**. 

Note that the probability values are for a prediction of positive sentiment (+1) given a specific value for featuere or input x and its weight/coefficient. so $$ P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) $$ 
We can easily find probability of a negative sentiment, just 1 - P(+1) !!

In [283]:
import numpy as np
probabilities = 1.0 * 1/(1 + np.exp(-scores))
print probabilities

[  9.96324003e-01   4.02795251e-02   2.96835318e-05]


**Checkpoint**: Make sure your probability predictions match the ones obtained from  scikit-learn.

In [284]:
print "Class predictions according to sklearn:" 
probs = sentiment_model.predict_proba(sample_test_matrix)
print probs[:,1] #look at values in 2nd column

Class predictions according to sklearn:
[  9.96324003e-01   4.02795251e-02   2.96835318e-05]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review? **third** cause it has lowest value to power of -5.

# Find the most positive (and negative) review

**13.** We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [282]:
#scores = sentiment_model.predict(test_data, output_type='probability')
#scores.topk(20, reverse=True)    
#scores in not sframe - actually closest to 'SArray' object but has no attribute 'topk'
#so we add prob predictions as extra col in test_data, then sort by this column and print 1st 20 rows

# also don't do reverse=True  for topk, as it will then grab lowest prob values into top of list!!
prob_pred = sentiment_model.predict_proba(test_matrix)
prob_pred =  prob_pred[:,1]
prob1 =  (sorted(prob_pred, reverse=True)[:20])
print prob1
#sorted(range(len(prob_pred)), key=lambda x: prob_pred[x])[-20:]
#topk = test_data.topk('prob_pred', 20)
#topk.print_rows(num_rows=20)
print np.argsort(prob_pred)[::-1][:20]   

#print test_data[ np.argsort(prob_pred)[::-1][:20], ]

print test_data[ [15732, 20743, 18112, 17558,  4140 , 9555, 26830, 30535 , 9125, 30634, 32782, 24899,
 21531, 11923, 14482, 25554 ,24286, 30076, 33060, 31271] ]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99999999999999978, 0.99999999999999978, 0.99999999999999956]
[15732 20743 18112 17558  4140  9555 26830 30535  9125 30634 32782 24899
 21531 11923 14482 25554 24286 30076 33060 31271]


TypeError: Invalid key type: must be str, bytes or type

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [29]:
topk = test_data.topk('prob_pred', 20,reverse=True)
topk.print_rows(num_rows=20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Jolly Jumper Arctic Sneak ... | I am a "research-aholic" i... |  5.0   |
| Levana Safe N'See Digital ... | This is the first review I... |  1.0   |
| Snuza Portable Baby Moveme... | I would have given the pro... |  1.0   |
| Fisher-Price Ocean Wonders... | We have not had ANY luck w... |  2.0   |
| VTech Communications Safe ... | This is my second video mo... |  1.0   |
| Safety 1st High-Def Digita... | We bought this baby monito... |  1.0   |
| Chicco Cortina KeyFit 30 T... | My wife and I have used th... |  1.0   |
| Prince Lionheart Warmies W... | *****IMPORTANT UPDATE*****... |  1.0   |
| Valco Baby Tri-mode Twin S... | I give one star to the dim... |  1.0   |
| Adiri BPA Free Natural Nur... | I will try to write an obj... |  2.0   |
| Munchkin Nursery Projec

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [142]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    '''
    Many ways to do it 
    1.Get margin or scores and predict which side of decision bounday each sample falls
    by looking at the sign (+ve/-ve) of the score for each data point
    
      scores = model.predict(data, output_type='margin')
      predicted_labels = [+1 if x > 0 else -1 for x in scores]
    
    2.Get probabilities that a given sample is a +ve setiment given sample and weights
    and if prob >= 0.5, assign +ve +1 sentiment, -ve -1 otherwise
    
      probs = model.predict(data, output_type='probability')
      predicted_labels = [+1 if x >= 0.5 else -1 for x in probs]
    
    3. Easiest - get prediction in form of class labels +1 or -1
    default output of model.predict is class predictions
    
      predicted_labels =  model.predict(data, output_type='class')  
      
    '''
    predicted_labels =  model.predict(data) 
    
    # Compute the number of correctly classified examples
    # understand why this will always give num of data points regardless True/False
    # 1st to element-wise boolean comparision between the 2 arrays 
    # and then count how many comparisions returned True, i.e same class in both array !! 
    # but doesn't work - in fact (predicted_labels == true_labels) outputs 1s and 0s!!!!
    print "num_correct using len()", len ( (predicted_labels == true_labels) == 'True' )
    print (predicted_labels == true_labels)[0:10]
    print "num_correct using sum()",(predicted_labels == true_labels).sum() 
    #sum gives correct result as same as counting # of 1s, 1+0+1+0+0=2!!
    
    # np.equal(x1,x2) -- > Return (x1 == x2) element-wise
    # Parameters:	 x1, x2 : array_like : Input arrays of the same shape.
    # Returns:	  out : ndarray or bool :  Output array of bools, or a single bool if x1 and x2 are scalars.  
    
    print np.equal(predicted_labels, true_labels)[0:10]
    num_correct = sum ( np.equal(predicted_labels, true_labels) )
    print "num_correct", num_correct
    print "num_data", len(data)
    
    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = num_correct * 1.0/len(data)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [143]:
print get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

num_correct using len() 33336
[1, 1, 0, 1, 1, 1, 1, 0, 1, 1]
num_correct using sum() 30487
[ True  True False  True  True  True  True False  True  True]
num_correct 30487
num_data 33336
0.914536837053


**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76). **0.91**

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better? **No**

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [144]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [145]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [34]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [146]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [147]:
print train_data[0]['word_count']

{'and': 3, 'love': 1, 'it': 3, 'highly': 1, 'osocozy': 1, 'bags': 1, 'holder': 1, 'leak': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'early': 1, 'not': 2, 'now': 1, 'disappointed': 1, 'wipe': 1, 'keps': 1, 'wise': 1, 'i': 1, 'planet': 1, 'my': 2, 'came': 1}


Since we are only working with a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [148]:
print train_data[0]['word_count_subset']

{'love': 1, 'disappointed': 1}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [149]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.338197     | 0.866815          |

| 6         | 7        | 0.398386     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.4174

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [150]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

num_correct using len() 33336
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
num_correct using sum() 28979
[ True  True  True  True  True  True  True False  True  True]
num_correct 28979
num_data 33336


0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [151]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552,0.0120888541331
word_count_subset,disappointed,1,-2.35509250061,0.0504149888557
word_count_subset,love,1,1.36543549368,0.0303546295109
word_count_subset,little,1,0.520628636025,0.0214691475665
word_count_subset,loves,1,1.67727145556,0.0482328275384
word_count_subset,product,1,-0.320555492996,0.0154311321362
word_count_subset,well,1,0.504256746398,0.021381300631
word_count_subset,great,1,0.94469126948,0.0209509926591
word_count_subset,easy,1,1.19366189833,0.029288869202
word_count_subset,work,1,-0.621700012425,0.0230330597946


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [152]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [170]:
print len (simple_model.coefficients[ simple_model.coefficients['value'] > 0 ]) - 1

10


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [177]:
positive_significant_words = simple_model.coefficients [simple_model.coefficients['value'] > 0 ]['index']
print positive_significant_words

simple_positive = simple_model.coefficients.sort('value', ascending=False)
simple_positive = simple_positive [simple_positive['value'] > 0 ]['index']

setimental_positive = sentiment_model.coefficients.sort('value', ascending=False)
sentiment_positive = setimental_positive [setimental_positive['value'] > 0 ]['index']

print len(simple_positive)
print len(sentiment_positive)
simple_positive = simple_positive.to_numpy()
sentiment_positive = sentiment_positive.to_numpy()
print "These words have positive weights in both the models\n", np.intersect1d(simple_positive, sentiment_positive)

[None, 'love', 'little', 'loves', 'well', 'great', 'easy', 'able', 'perfect', 'old', 'car', ... ]
11
68419
These words have positive weights in both the models
[None 'able' 'car' 'easy' 'great' 'little' 'love' 'loves' 'old' 'perfect'
 'well']


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [178]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

num_correct using len() 133416
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
num_correct using sum() 130673
[ True  True  True  True  True  True  True  True  True  True]
num_correct 130673
num_data 133416


0.97944024704683097

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [179]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

num_correct using len() 133416
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
num_correct using sum() 115647
[ True  True  True  True  True  True  True  True  True  True]
num_correct 115647
num_data 133416


0.86681507465371466

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set? **sentiment_model**

Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [180]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

num_correct using len() 33336
[1, 1, 0, 1, 1, 1, 1, 0, 1, 1]
num_correct using sum() 30487
[ True  True False  True  True  True  True False  True  True]
num_correct 30487
num_data 33336


0.91453683705303579

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [181]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

num_correct using len() 33336
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
num_correct using sum() 28979
[ True  True  True  True  True  True  True False  True  True]
num_correct 28979
num_data 33336


0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set? **sentiment_model**

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [182]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [185]:
num_positive  = (test_data['sentiment'] == +1).sum()
num_negative = (test_data['sentiment'] == -1).sum()
print num_positive
print num_negative
print round(num_positive/len(test_data), 2)

28095
5241
0.84


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)? Better 91% with train data, but with test data accuracy was 87%. Baseline accuracy from majority class classfier is 84%. So we can't say that sentiment_model is definitely better than baseline, as its accuracy is only 3% better.